# IN this notebook i will be using SVM layer instead of Logistic regression layer for final output and let's ee the results.
# Upvote the notebook if you liked it.
# In this notebook i am using my previous notebook https://www.kaggle.com/atulyaatul1999/hackerearth-tis-still-the-season-to-be-jolly

## I am using approch of krish naik sir, his youtube video link is https://www.youtube.com/watch?v=upo34AT1Cys

## Basically the main concept behind this approach is that we are using deep learning for feature extraction and at the end using SVM model for classification

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading data

In [ ]:
import pandas as pd

In [ ]:
train_df=pd.read_csv('../input/hackereath-holiday-season-deep-learning-contest/dataset/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df['Class'].value_counts()

In [ ]:
from keras.preprocessing import image

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
img=image.load_img('../input/hackereath-holiday-season-deep-learning-contest/dataset/train/image1000.jpg')

In [ ]:
img=image.img_to_array(img)/255

In [ ]:
plt.imshow(img)

In [ ]:
print(img.shape)

# Dividing data in valid and train part using StratifiedKFold as data is skewed

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_df['Kfold']=-1

In [ ]:
train_df.head()

In [ ]:
train_df=train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df.tail()

In [ ]:
y=train_df['Class']

In [ ]:
kf=StratifiedKFold(n_splits=5)

In [ ]:
for f,(t_,v_) in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[v_,'Kfold']=f

In [ ]:
train_df.head()

In [ ]:
train=train_df[train_df['Kfold']!=4]

In [ ]:
valid=train_df[train_df['Kfold']==4]

In [ ]:
valid.tail()

In [ ]:
valid['Class'].value_counts()

# As the data is large so it will be better to use datagenerator, so I am using keras Imagedatagenerator and we can do data augmentation in this step

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         width_shift_range=0.1,
#         height_shift_range=0.1)

train_generator=train_datagen.flow_from_dataframe(dataframe=train,
                                            directory="../input/hackereath-holiday-season-deep-learning-contest/dataset/train/",
                                            x_col="Image",
                                            y_col="Class",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(331,331))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

valid_datagen = ImageDataGenerator(rescale=1./255)
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         width_shift_range=0.1,
#         height_shift_range=0.1)

valid_generator=valid_datagen.flow_from_dataframe(dataframe=valid,
                                            directory="../input/hackereath-holiday-season-deep-learning-contest/dataset/train/",
                                            x_col="Image",
                                            y_col="Class",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(331,331))

# Loading nasnet large model and setting all layers except last 35 as non trainable so we can generalise our model on our data and also do transfer learning for better results

In [ ]:
from keras.applications.nasnet import NASNetLarge
# from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [ ]:
nasnet=NASNetLarge(include_top=True,weights='imagenet')

In [ ]:
import tensorflow as tf

In [ ]:
x=nasnet.layers[-2].output
fc1=Dense(6, kernel_regularizer=tf.keras.regularizers.l2(0.02),activation='softmax')(x)

In [ ]:
my_model=Model(inputs=nasnet.input,outputs=fc1)

In [ ]:
# my_model.summary()

In [ ]:
from keras.optimizers import Adam

In [ ]:
adam=Adam(learning_rate=0.0001)

In [ ]:
for l in my_model.layers[:-5]:
    #print(l)
    l.trainable = False
my_model.compile(optimizer='adam',loss ='squared_hinge',metrics=["accuracy"])

In [ ]:
my_model.fit_generator(train_generator,steps_per_epoch=5176//128,validation_data=valid_generator,validation_steps=1293//128,epochs=4)

In [ ]:
my_model.save('model.h5')

# If you liked the notebook please upvote.
